In [1]:
!pip install nltk

In [2]:
import pandas as pd
import random
from azureml.core import Workspace, Datastore, Dataset
from nltk.tokenize import WhitespaceTokenizer


In [3]:
df = pd.read_csv("~/cloudfiles/code/Users/capstonesipsml/Data/Alpha_Dataset.csv", names=["Data", "Label"], header=0)

usernames_df = pd.read_csv("~/cloudfiles/code/Users/mitchall.boesel/Twitter/tweets.csv", names=["Data", "Label"], header=0)
hostnames_df = pd.read_csv("~/cloudfiles/code/Users/caesar.white/hostnames_strings.csv", names=["Data", "Label"], header=0)
filenames_df = pd.read_csv("~/cloudfiles/code/Users/jongyun.kim/featuriaztion/filenames.csv", names=["Data", "Label"], header=0)
ip_df = pd.read_csv("~/cloudfiles/code/Users/cougsInAzure/IP_Datapoints.csv", names=["Data", "Label"], header=0)
servernames_df = pd.read_csv("~/cloudfiles/code/Users/mitchall.boesel/servernames.csv", names=["Data", "Label"], header=0)

In [6]:
# Importing everyone's datasets

usernames_df = pd.read_csv("../../../Dataset_CSVs/tweetsv2.csv", names=["Data", "Label"], header=0)
hostnames_df = pd.read_csv("../../../Dataset_CSVs/hostname_stringsv2.csv", names=["Data", "Label"], header=0)
filenames_df = pd.read_csv("../../../Dataset_CSVs/filenamesv2.csv", names=["Data", "Label"], header=0)
ip_df = pd.read_csv("../../../Dataset_CSVs/ip_datapointsv2.csv", names=["Data", "Label"], header=0)
servernames_df = pd.read_csv("../../../Dataset_CSVs/servernamesv2.csv", names=["Data", "Label"], header=0)


In [9]:
ip_df['Label'].value_counts()

1    2527
0    2473
Name: Label, dtype: int64

In [7]:
l = [usernames_df,hostnames_df,filenames_df,ip_df,servernames_df]
for d in l:
    print(d['Label'].value_counts())

username    7319
nothing     3009
Name: Label, dtype: int64
hostname    5025
nothing     4975
Name: Label, dtype: int64
filename    8741
Name: Label, dtype: int64
IP         2527
nothing    2473
Name: Label, dtype: int64
servername    186
Name: Label, dtype: int64


In [34]:
# Combining datasets and randomizing resulting list

dataset = usernames_df.values.tolist() + hostnames_df.values.tolist() + filenames_df.values.tolist() + ip_df.values.tolist() + servernames_df.values.tolist()
random.shuffle(dataset)
print(len(dataset))


34255


In [8]:
# Features

# Username specific features
def LowerUnderscoreUpper(s):
    l = len(s)

    for i in range(0,l-2):
        if s[i].islower() and s[i+1] == '_' and s[i+2].isupper():
            return True
    return False

def HasUnderscore(s):
    s = str(s)

    for c in s:
        if c == '_':
            return True
    return False

def LowerUpperLower(s):
    s = str(s)

    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            return True
    return False

def MultipleLowerUpperLower(s):
    s = str(s)

    flag = False
    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            if flag:
                return True
            else:
                flag = True
    return False

def ExactlyTwoUppercase(s):
    count = 0

    for c in s:
        if c.isupper():
            count+=1
    if count ==2:
        return True
    else:
        return False

def AllLowerMoreThan(bound, s):
    s = str(s)

    if len(s) < bound:
        return False
    for c in s:
        if c.isupper():
            return False
    return True

def AdjacentUppers(s):
    s = str(s)

    for i in range(0,len(s)-1):
        if s[i].isupper() and s[i+1].isupper():
            return True
    return False

def StartLetterEndNonLetter(s):
    s = str(s)

    if s[0].isalpha() and not s[-1].isalpha():
        return True
    return False

def IsUrl(s):
    s = str(s)
    l = len(s)
    if l > 3:
        if s[0:4] == "http":
            return True
    if l > 4:
        if s[0:5] == "https":
            return True
    return False

# Hostname specific features
def LengthGTLT(gt, ls, s):
    l = len(str(s))
    if l >= gt and l <= ls:
        return True
    else:
        return False

def IllegalHostnameChars(s):
    s = str(s)
    illegal_chars = [".", "\", ""/", "*", "?", "\"", "<", ">", "|", ",", "~", ":", "!", "@", "#", "$", "%", "^", "&", "'", "(", ")", "{", "}", " "]
    for c in s:
        if c in illegal_chars:
            return True
    return False

def AlphaOrDigit(s):
    s = str(s)
    if s.isalpha() or s.isdigit():
        return True
    return False

def HostIllegalEnding(s):
    s = str(s)
    if s[-1] == '-' or s[-1] == '.':
        return True
    return False

# Filename specific features

def ContainsPeriod(s):
    s = str(s)
    for c in s:
        if c == '.':
            return True
    return False


def HasSlash(s):
    s = str(s)
    for c in s:
        if c == '/' or c == '\\':
            return True
    return False

def HasMultipleSlash(s):
    s = str(s)
    slashes = ['/','\\']
    flag = False
    for c in s:
        if not flag and c in slashes:
            flag = True
        elif flag and c in slashes:
            return True
    return False

def HasPossibleExtension(s):
    s = str(s)
    for i in range(0,len(s)):
        if s[i] == '.':
            x = s[i:-1]
            if len(x) >= 2 and len(x) <= 4:
                return True
    return False

# IP Features
def NumbersThenPeriod(s):
    s = str(s)
    freq = 0
    for i in range(0,len(s)-1):
        if s[i] >= '0' and s[i] <= '9' and s[i+1] == '.':
            freq += 1
    return True if freq == 3 else False

def AtLeastFourDigits(s):
    s = str(s)
    counter = 0
    for c in s:
        if c.isnumeric():
            counter += 1
    return True if counter >=4 else False

# Servername Features

def HasPeriodAndSlash(s):
    s = str(s)
    period = False
    slash = False
    for c in s:
        if c == '.':
            period = True
        if s =='/':
            slash = True
    return True if period and slash else False

def HasInstanceNumPeriod(s):
    s = str(s)
    for i in range(0,len(s)-1):
        if s[i].isnumeric() and s[i+1] == '.':
            return True
    return False

def HasMultipleNumPeriod(s):
    s = str(s)
    flag = False
    for i in range(0,len(s)-1):
        if s[i].isnumeric() and s[i+1] == '.' and not flag:
            flag = True
        if s[i].isnumeric() and s[i+1] == '.' and flag:
            return True
    return False

# List of the feature names, for resulting csv file column names
'''
feature_names =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",
    "AdjacentUppers", "StartLetterEndNonLetter", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",
    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", 
    "HasMultipleNumPeriod", "Label"]
'''
feature_names =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",
    "AdjacentUppers", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",
    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", 
    "HasMultipleNumPeriod", "Label"]

In [77]:
# extracting features and labels
feature_list = []
 
tk = WhitespaceTokenizer() 

for dp in dataset:
    d = str(dp[0])
    label = dp[1]
    tokens = tk.tokenize(d)
    tokens = list(filter(lambda x: not IsUrl(x), tokens))
    dp_list = []
    dp_list.append(True) if True in list(map(lambda x: LowerUnderscoreUpper(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasUnderscore(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: MultipleLowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ExactlyTwoUppercase(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AllLowerMoreThan(8,x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AdjacentUppers(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: StartLetterEndNonLetter(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LengthGTLT(1, 8, x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: IllegalHostnameChars(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AlphaOrDigit(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HostIllegalEnding(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ContainsPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasMultipleSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasPossibleExtension(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: NumbersThenPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AtLeastFourDigits(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasPeriodAndSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasInstanceNumPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasMultipleNumPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(label)
    feature_list.append(dp_list)
random.shuffle(feature_list)

In [73]:
import pandas as pd
import numpy as np
import json
import ast
from nltk.tokenize import WhitespaceTokenizer
import random
from azureml.core import Workspace, Dataset
import math

In [78]:
# Splitting the dataset into train and test

#X = list(map(lambda x: x[:-1], dataset))
#y = list(map(lambda x: x[-1], dataset))

X = list(map(lambda x: x[:-1], feature_list))
y = list(map(lambda x: x[-1], feature_list))
length = len(feature_list)
interval = math.floor(length/3)
y_test = y[:interval]
y_train = y[interval:]

X_test = X[:interval]
X_train = X[interval:]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
print(len(X_test))
print(len(X_train))

11418
22837


In [80]:
# Functions to compute different metrics

def calcAccuracy(target, predictions):
    total = len(predictions)
    correct = 0
    for i in range(len(predictions)):
        if target[i] == predictions[i]:
            correct += 1
        accuracy = float(correct) / float(total)
    return accuracy
'''
def FalsePositivesMultiClass(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 'nothing' and pred[i] != 'nothing':
            false_positives.append(x_test[i])
    return false_positives

def FalseNegativesMultiClass(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] != 'nothing' and pred[i] == 'nothing':
            false_negatives.append(x_test[i])
    return false_negatives
'''
def FalsePositivesMultiClass(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 'nothing' and pred[i] != 'nothing':
            false_positives.append(dataset[i])
    return false_positives

def FalseNegativesMultiClass(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] != 'nothing' and pred[i] == 'nothing':
            false_negatives.append(dataset[i])
    return false_negatives


def MislabeledMultiClass(x_test, y_test, pred):
    mislabeled = []
    for i in range(0,len(x_test)):
        if y_test[i] != 'nothing' and pred[i] != 'nothing' and y_test[i] != pred[i]:
            mislabeled.append(x_test[i])
    return mislabeled

def FalsePositives(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 0 and pred[i] == 1:
            false_positives.append(x_test[i])
    return false_positives

def FalseNegatives(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] != 1 and pred[i] == 0:
            false_negatives.append(x_test[i])
    return false_negatives

def PrintResults(classifier, acc, recall, precision):
    print(classifier + " Classifier")
    print("Accuracy= ", acc)
    print("Precision= ", precision)
    print("Recall= ", recall)

In [15]:
# BEGIN MACHINE LEARNING MODELS
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [81]:
# DECISION TREE CLASSIFIER
dt = DecisionTreeClassifier().fit(X_train, y_train)
pred = dt.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred,average='macro')
#precision = precision_score(y_test, pred, average='macro')
PrintResults("Decision Tree", acc, "N/A", "N/A")


false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)

print(len(false_negatives))
for row in false_negatives:
    print(row)

print(len(false_negatives))

Decision Tree Classifier
Accuracy=  0.9523559292345419
Precision=  N/A
Recall=  N/A
[["shonda510 You're in!We'll send you a daily reminder to DUB THE VOTE right here on Twitter.Remember to vote on https://t.co/NAlxef2FQY too!Reply #stop to opt out of #NBAAllStar reminders", 'username'], ['ELIZABETHBROWNELIZABETHBROWNEL', 'nothing'], ['/style2.css', 'filename'], ['\\!|', 'nothing'], ['groucho-singapore', 'hostname'], ['groucho-eu', 'hostname'], ['|!^', 'nothing'], ['/presentations/logstash-puppetconf-2012/images/sysadvent.png', 'filename'], ['RT melodieaikels: Dinosaurs is on Disney Plus Yall! Youre Welcome!! https://t.co/yCp9TT1Hr2', 'username'], ['Request sent at 2013-06-02 23:40:49', 'nothing'], ['RT AcerUK_Official: We are teaming up with your favourite gaming influencers to give you the chance to win a ton of Predator gaming gear!', 'username'], ['Youre asking me to be like an oracle of the future, says Campbell Harvey, a Duke international business professor who also works as a pa

In [83]:
print(len(false_negatives))


for row in false_negatives:
    print(row)

#print(len(false_negatives))

300
['72145145219', 'nothing']
['RT DirkErik_Schulz: Disney hates his ducks. https://t.co/SzNqBOURSg', 'username']
['The author, Eugene Rogan, judiciously deals with the Armenian Genocide issue and explains it. He calls it by what it was, a Genocide. I have serious doubts Luke would agree with this considering his love for seeing everything through a skewed, geopolitically motivated agenda... https://t.co/LzR4NKPvUf', 'nothing']
['IP address =213.248.126.87', 'IP']
['Destination IP: 199.192.159.238', 'IP']
['Blackhawks ML  Nuggets ML  Raptors ML  Parlay (+397) 5 units #GamblingTwitter #FreePicks #HoosierGamblers', 'nothing']
['8TERRELL-FIGUEROA', 'nothing']
['MARCELLALINDSEYMARCELLALINDSEYMA', 'nothing']
['groucho-sa', 'hostname']
['!REGINA-BERRY', 'nothing']
['groucho-sa', 'hostname']
['Client port number: 47128', 'nothing']
["Microsoft Wants To Make A Chatbot That Could Let Us Talk' To Dead Loved Ones https://t.co/BOpv2LfpSY https://t.co/4gr2ENfHff", 'nothing']
['!$>', 'nothing']
['R

In [ ]:
dt.predict(X_test)